In [1]:
import pandas as pd
import sqlalchemy as sqla
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id='3b8a7beb4a434db98620cfae1983cffc',client_secret='a5bc3f14c035496690b540b81c5053f5',))
from scipy.stats import spearmanr

In [2]:
##Setting Up SQL Engine and Extracting Data
engine = sqla.create_engine('sqlite:///top200data')
pd.read_csv('../data/top200data.csv').to_sql('top200data', engine)

ValueError: Table 'top200data' already exists.

In [4]:
songslist = pd.read_sql('SELECT DISTINCT song, songcode, artist FROM top200data WHERE song & songcode IS NOT NULL ',engine)

In [5]:
def searchfeatures(x,feature):
    '''
    Returns specific feature of a song from Spotify API
    
    Inputs: x=song code, feature: feature of song
    Output: Value of specific feature for the song, nan if no value found
    '''
    try:
        return spotify.audio_features(x)[0][feature]
    except:
        return np.nan
    

In [ ]:
#Testing Code on Smaller Dataframe
testdf=pd.DataFrame(columns=['track','dance'])
testdf['track']=songslist['songcode'][0:5]
testdf['dance']=testdf['track'].apply(lambda x: searchfeatures(x,'danceability') )
testdf['energy']=testdf['track'].apply(lambda x: searchfeatures(x,'energy') )
testdf['acousticness']=testdf['track'].apply(lambda x: searchfeatures(x,'acousticness') )

In [ ]:
##Filling Features of Each Song to Dataframe
songslist['danceability'] = songslist['songcode'].apply(lambda x: searchfeatures(x,'danceability')) 
songslist['acousticness'] = songslist['songcode'].apply(lambda x: searchfeatures(x,'acousticness')) 
songslist['energy'] = songslist['songcode'].apply(lambda x: searchfeatures(x,'energy')) 

In [4]:
##Only Main Artist for Collaborations
songslist['artistmain'] = songslist['artist'].apply(lambda x: x.split(', ')[0])

In [ ]:
##Finding Correlations between Features (Danceability, Energy, Acousticness) and Streams for Dates
df_all = pd.read_sql('select * from top200data',engine)
corr2_df = pd.DataFrame(columns = ["date", "corr_dance", "corr_energy", "corr_acousticness"])
for i in range(len(df_all["date"].unique())):
    tempdf = pd.DataFrame(columns = ['index', 'id', 'rank', 'song_x', 'date', 'artist_x', 'regioncode','songcode', 'streams', 'Unnamed: 0', 'Unnamed: 0.1', 'song_y','artist_y', 'danceability', 'energy', 'acousticness'])
    for j, region in enumerate(df_all["regioncode"].unique()):
        readdf = pd.read_sql(f'select * from top200data where regioncode = "{region}" AND date = "{ls_date[i]}"',engine)
        merge_temp = pd.merge(readdf, songslist, on= "songcode")
        tempdf = tempdf.append(merge_temp, ignore_index=True)
    temp_dance = spearmanr(tempdf["streams"] , tempdf["danceability"])[0]
    temp_energy = spearmanr(tempdf["streams"] , tempdf["energy"])[0]
    temp_acou = spearmanr(tempdf["streams"] , tempdf["acousticness"])[0]
    corr2_df.loc[i] = [f"{ls_date[i]}", temp_dance , temp_energy, temp_acou ]

In [38]:
##Export Datsets
songslist.to_csv('../data/songsdata.csv')
corr2_df.to_csv("../data/correlationsdata.csv")

In [10]:
pd.read_csv("../data/correlationsdata.csv")

,Unnamed: 0,Unnamed: 0.1,date,corr_dance,corr_energy,corr_acousticness
0,0,0,2017-01-06,-0.139048,-0.046807,0.019445
1,1,1,2017-01-13,-0.113154,-0.038381,0.015422
2,2,2,2017-01-20,-0.123196,-0.033918,0.013208
3,3,3,2017-01-27,-0.113097,-0.058542,0.014405
4,4,4,2017-02-03,-0.117659,-0.073788,0.029335
...,...,...,...,...,...,...
203,203,203,2020-11-27,-0.063788,0.006520,0.019311
204,204,204,2020-12-04,-0.097027,-0.022224,0.014233
205,205,205,2020-12-11,-0.078633,-0.002469,-0.004289
206,206,206,2020-12-18,-0.103611,-0.007120,0.013577
